In [51]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
from netCDF4 import Dataset as NetCDFFile
from matplotlib.colors import ListedColormap,LinearSegmentedColormap,Normalize
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
carpeta='MUNDO'
zona='MUNDO'

x1=0
x2=360
y1=0
y2=180
    
t1=0
t2=60


for t in range(t1,t2):
    ncfile2 = NetCDFFile("/Users/jonre/datos_fijos.nc", "r")
    ncfile = NetCDFFile("/Users/jonre/datos_2015_2019.nc", "r")
    lat  = ncfile.variables["latitude"][y1:y2]
    lon  = ncfile.variables["longitude"][x1:x2]
    z=(ncfile2.variables["z"][0,y1:y2,x1:x2])
    lsm=(ncfile2.variables["lsm"][0,y1:y2,x1:x2])
    runoff=(ncfile.variables["ro"][t,0,y1:y2,x1:x2])
    total_column_water=(ncfile.variables["tcwv"][t,0,y1:y2,x1:x2])
    E=(ncfile.variables["e"][t,0,y1:y2,x1:x2])
    precip=(ncfile.variables["tp"][t,0,y1:y2,x1:x2])
    Qx=(ncfile.variables["p71.162"][t,0,y1:y2,x1:x2])
    Qy=(ncfile.variables["p72.162"][t,0,y1:y2,x1:x2])

    R=6371000    

    lat2=np.delete(lat, [0])
    lat2=lat2+0.5 #89.5,88.5,...,-89.5
    lon2=np.delete(lon,[0])
    lon2=lon2-0.5

    def mask():
        valor=np.zeros((len(lat),len(lon)))
        for y in range(len(lat)):
            for x in range(len(lon)):
                if ((z[y,x]/9.80665)<26.0)&(lsm[y,x]<0.25):
                    valor[y,x]=0.0
                else:
                    valor[y,x]=1.0
        return valor

#mascara2 lon: 359, lat:180 BIEN
    def mask2():

        m=mask()            
        #mmask=np.zeros((len(lat2),len(lon2)))
        mmask=(m[0:-1,0:-1]+m[1:,0:-1]+m[0:-1,1:]+m[1:,1:])/4.
    
        for y in range(len(lat2)):
            for x in range(len(lon2)):
                mmask[y,x]=(m[y,x]+m[y+1,x]+m[y,x+1]+m[y+1,x+1])/4.0
                if mmask[y,x]>0.5:
                    mmask[y,x]=1
                else:
                    mmask[y,x]=0
        return mmask 

    def sacar_Eflow():
        Eflow = np.zeros((len(lat2),len(lon2)))
        Eflow_mm = np.zeros((len(lat2),len(lon2)))
        P= np.zeros((len(lat2),len(lon2)))
        P_mm= np.zeros((len(lat2),len(lon2)))
        Column_water= np.zeros((len(lat2),len(lon2)))
        Runoff= np.zeros((len(lat2),len(lon2)))
        Runoff_mm= np.zeros((len(lat2),len(lon2)))
        EvapPrec = np.zeros((len(lat2),len(lon2)))
        mask=mask2()
        for y in range(len(lat2)): 
            for x in range(len(lon2)):
                Atetha=math.radians(1.0)
                Alambda=math.radians(1.0)
                tethamedio=math.radians(lat2[y])
                #A=(R**2)*(math.sin(tethamedio+(Atetha/2.0))-math.sin(tethamedio-(Atetha/2.0)))*Alambda
                A=(R**2)*2*math.cos(tethamedio)*math.sin(Atetha/2)*Alambda
            
                if mask[y,x]==1.0: #si la mask es 0 no lo tenemos en cuenta
                    Eflow[y,x]=(E[y,x] +E[y+1,x] + E[y,x+1] +E[y+1,x+1] )*(A*1000)/(86400*4) #kg/s
                    Eflow_mm[y,x]=(E[y,x] +E[y+1,x] + E[y,x+1] +E[y+1,x+1] )*(1000*30)/(4) #mm/month
                    P[y,x]=(precip[y,x] +precip[y+1,x] + precip[y,x+1] +precip[y+1,x+1])*(A*1000)/(86400*4)
                    P_mm[y,x]=(precip[y,x] +precip[y+1,x] + precip[y,x+1] +precip[y+1,x+1])*(1000*30)/(4)
                    Runoff[y,x]=(runoff[y,x] +runoff[y+1,x] + runoff[y,x+1] +runoff[y+1,x+1])*(A*1000)/(86400*4)
                    Runoff_mm[y,x]=(runoff[y,x] +runoff[y+1,x] + runoff[y,x+1] +runoff[y+1,x+1])*(1000*30)/(4)
                    Column_water[y,x]=(total_column_water[y,x] +total_column_water[y+1,x] + total_column_water[y,x+1] +total_column_water[y+1,x+1])*A/(30*86400*4)
                    EvapPrec[y,x]=-1*Eflow_mm[y,x]-P_mm[y,x]
                
        return Eflow, Eflow_mm, P, P_mm, Runoff, Runoff_mm, Column_water, EvapPrec
    

    Eflow, Eflow_mm, P, P_mm, Runoff, Runoff_mm, Column_water, EvapPrec =sacar_Eflow()
    
    
    np.savetxt('C:/Users/jonre/TFG/%s/mask_%s.txt'%(carpeta,zona), mask())
    np.savetxt('C:/Users/jonre/TFG/%s/mask2_%s.txt'%(carpeta,zona), mask2())
    np.savetxt('C:/Users/jonre/TFG/%s/lon_%s.txt'%(carpeta,zona), lon)
    np.savetxt('C:/Users/jonre/TFG/%s/lat_%s.txt'%(carpeta,zona), lat)
    np.savetxt('C:/Users/jonre/TFG/%s/lon2_%s.txt'%(carpeta,zona), lon2)
    np.savetxt('C:/Users/jonre/TFG/%s/lat2_%s.txt'%(carpeta,zona), lat2)
    np.savetxt('C:/Users/jonre/TFG/%s/Eflow_%s_%d.txt'%(carpeta,zona,t), -1*Eflow)
    np.savetxt('C:/Users/jonre/TFG/%s/Eflow_mm_%s_%d.txt'%(carpeta,zona,t), -1*Eflow_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/P_%s_%d.txt'%(carpeta,zona,t), P)
    np.savetxt('C:/Users/jonre/TFG/%s/P_mm_%s_%d.txt'%(carpeta,zona,t), P_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/Runoff_mm_%s_%d.txt'%(carpeta,zona,t), Runoff_mm)
    np.savetxt('C:/Users/jonre/TFG/%s/Column_water_%s_%d.txt'%(carpeta,zona,t), Column_water)
    np.savetxt('C:/Users/jonre/TFG/%s/Qx_%s_%d.txt'%(carpeta,zona,t), Qx)
    np.savetxt('C:/Users/jonre/TFG/%s/Qy_%s_%d.txt'%(carpeta,zona,t), Qy)
    np.savetxt('C:/Users/jonre/TFG/%s/Evap_Precip_%s_%d.txt'%(carpeta, zona,t), EvapPrec)
    


#------

    ncfile.close()
    ncfile2.close()
print('acabado')

acabado
